# Tutorial 21: Plan-and-Execute Pattern

In this tutorial, you'll build a **plan-and-execute pattern** for complex multi-step tasks with adaptive replanning.

**What you'll learn:**
- How to break complex tasks into step-by-step plans
- Executing plans sequentially with context from previous steps
- Implementing replanning to adapt based on execution results
- When to use plan-execute vs pure ReAct agents
- Combining planning with tool-enabled execution

By the end, you'll have a working plan-and-execute system that plans, executes, and adapts.

## Prerequisites

- Completed Tutorial 02 (Tool Calling and ReAct)
- Understanding of multi-step reasoning
- Ollama running with a capable model (llama3.1:8b or larger recommended)

## Why Plan-and-Execute?

ReAct agents (Tutorial 02) make step-by-step decisions without a global plan. This works well for simple tasks but has limitations:

1. **No Lookahead**: Can't optimize the overall approach
2. **Token Inefficient**: Every step requires full context
3. **Hard to Debug**: Can't see the intended strategy upfront
4. **Resource Waste**: Uses expensive model for every micro-decision

**Plan-and-Execute advantages:**
- Explicit plan visible before execution
- Can use different models for planning vs execution
- Better for multi-step tasks requiring coordination
- Easier to debug and validate
- Supports replanning when needed

```
┌─────────┐     ┌──────────┐     ┌───────────┐
│ Planner │────►│ Executor │────►│ Replanner │
└─────────┘     └──────────┘     └─────┬─────┘
                      ↑                 │
                      │    New Plan     │ Done?
                      └─────────────────┤
                                        │
                                        ▼
                                   ┌────────┐
                                   │  END   │
                                   └────────┘
```

## Step 1: Setup and Verify Environment

In [ ]:
# Verify our setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama URL: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")
print("Setup verified!")

## Step 2: Define the Plan-Execute State

Our state tracks:
- The original task
- The current plan (list of steps)
- Past executed steps with results
- Current step index
- Final response

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
import operator


class PlanExecuteState(TypedDict):
    """State schema for plan-and-execute pattern."""
    
    # Original task to accomplish
    task: str
    
    # Current plan (list of step descriptions)
    plan: list[str]
    
    # History of (step, result) pairs - uses operator.add to accumulate
    past_steps: Annotated[list[tuple[str, str]], operator.add]
    
    # Index of current step to execute
    current_step: int
    
    # Final response when complete
    response: str


print("State defined!")
print("\nKey fields:")
print("- plan: List of step descriptions")
print("- past_steps: Accumulated (step, result) using operator.add")
print("- current_step: Tracks progress through plan")
print("- response: Final answer when task is complete")

## Step 3: Create the Planner Node

The planner analyzes the task and creates a step-by-step plan.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage


class Plan(BaseModel):
    """Structured plan output."""
    
    steps: list[str] = Field(
        description="List of 3-7 actionable steps to accomplish the task"
    )


PLANNER_PROMPT = """You are a strategic planner that breaks down complex tasks into clear, actionable steps.

Your responsibilities:
1. Analyze the task thoroughly
2. Create a step-by-step plan with 3-7 concrete steps
3. Ensure each step is specific and actionable
4. Order steps logically for efficient execution
5. Make steps independent where possible

Guidelines:
- Keep steps simple and focused
- Avoid vague or abstract steps
- Each step should have a clear completion criterion
- Consider dependencies between steps
- Aim for the minimum number of steps needed"""


def create_planner_node(llm):
    """Create a planner that generates step-by-step plans."""
    
    structured_llm = llm.with_structured_output(Plan)
    
    def planner(state: PlanExecuteState) -> dict:
        messages = [
            SystemMessage(content=PLANNER_PROMPT),
            HumanMessage(content=f"""Task: {state['task']}

Create a detailed step-by-step plan to accomplish this task.
Break it into 3-7 clear, actionable steps.""")
        ]
        
        output = structured_llm.invoke(messages)
        steps = output.steps
        
        print(f"\n[Planner] Created plan with {len(steps)} steps:")
        for i, step in enumerate(steps, 1):
            print(f"  {i}. {step}")
        
        return {"plan": steps, "current_step": 0}
    
    return planner


print("Planner node creator defined!")

## Step 4: Create the Executor Node

The executor processes steps one at a time, building on previous results.

In [ ]:
EXECUTOR_PROMPT = """You are a task executor working through a plan step by step.

Your responsibilities:
- Execute the current step thoroughly
- Use available tools when needed
- Provide clear, specific results
- Build on previous step results when relevant

Focus on completing your assigned step effectively."""


def create_executor_node(llm, tools=None):
    """Create an executor that processes one step at a time."""
    
    # If tools provided, use ReAct agent
    if tools:
        from langgraph.prebuilt import create_react_agent
        react_agent = create_react_agent(llm, tools)
        use_tools = True
    else:
        use_tools = False
    
    def executor(state: PlanExecuteState) -> dict:
        plan = state.get("plan", [])
        current_step = state.get("current_step", 0)
        past_steps = state.get("past_steps", [])
        task = state["task"]
        
        # Check if we're done
        if current_step >= len(plan):
            return {}
        
        step = plan[current_step]
        
        # Build context from past steps
        context_parts = [f"Original task: {task}\n"]
        
        if past_steps:
            context_parts.append("Steps completed so far:")
            for i, (prev_step, prev_result) in enumerate(past_steps, 1):
                context_parts.append(f"\n{i}. {prev_step}")
                result_preview = prev_result[:200] + "..." if len(prev_result) > 200 else prev_result
                context_parts.append(f"   Result: {result_preview}")
            context_parts.append("\n")
        
        context = "".join(context_parts)
        
        print(f"\n[Executor] Step {current_step + 1}/{len(plan)}: {step[:60]}...")
        
        # Execute the step
        if use_tools:
            # Use ReAct agent with tools
            agent_input = {
                "messages": [
                    HumanMessage(content=f"""{context}
Now execute this step: {step}

Use tools if needed to complete this step thoroughly.""")
                ]
            }
            agent_result = react_agent.invoke(agent_input)
            messages = agent_result.get("messages", [])
            if messages:
                result = messages[-1].content
            else:
                result = "Step executed"
        else:
            # Use LLM without tools
            messages = [
                SystemMessage(content=EXECUTOR_PROMPT),
                HumanMessage(content=f"""{context}
Now execute this step: {step}

Provide a clear, specific result for this step.""")
            ]
            response = llm.invoke(messages)
            result = response.content
        
        print(f"[Executor] Completed")
        
        return {
            "past_steps": [(step, result)],
            "current_step": current_step + 1,
        }
    
    return executor


print("Executor node creator defined!")

## Step 5: Create the Replanner Node

The replanner reviews completed steps and decides whether to finalize or create a new plan.

In [ ]:
from typing import Union


class Response(BaseModel):
    """Final response when task is complete."""
    
    response: str = Field(
        description="Comprehensive response addressing the original task"
    )


class Act(BaseModel):
    """Action decision - either respond or create new plan."""
    
    action: Union[Response, Plan] = Field(
        description="Either a final Response or a new Plan for continued execution"
    )


REPLANNER_PROMPT = """You are a replanner that decides whether to continue with the plan or finalize the response.

Your responsibilities:
1. Review completed steps and their results
2. Determine if the original task is accomplished
3. Decide to either:
   - Respond with the final answer if the task is complete
   - Create a new plan if more work is needed

Guidelines:
- Only finalize if the task is truly complete
- If replanning, create steps that build on what's been done
- Be efficient - don't add unnecessary steps"""


def create_replanner_node(llm):
    """Create a replanner that decides to finalize or create new plan."""
    
    structured_llm = llm.with_structured_output(Act)
    
    def replanner(state: PlanExecuteState) -> dict:
        task = state["task"]
        past_steps = state.get("past_steps", [])
        
        # Format past steps for review
        steps_summary = []
        for i, (step, result) in enumerate(past_steps, 1):
            result_preview = result[:300] + "..." if len(result) > 300 else result
            steps_summary.append(f"{i}. {step}\n   Result: {result_preview}")
        
        steps_text = "\n\n".join(steps_summary)
        
        print(f"\n[Replanner] Reviewing {len(past_steps)} completed steps...")
        
        messages = [
            SystemMessage(content=REPLANNER_PROMPT),
            HumanMessage(content=f"""Original task: {task}

Steps completed:
{steps_text}

Based on these results, decide:
1. If the task is complete, provide a final Response synthesizing the results
2. If more work is needed, provide a new Plan with additional steps

What should we do next?""")
        ]
        
        output = structured_llm.invoke(messages)
        
        if isinstance(output.action, Response):
            print("[Replanner] Task complete, finalizing response")
            return {"response": output.action.response}
        else:  # New Plan
            print(f"[Replanner] Creating new plan with {len(output.action.steps)} steps")
            return {
                "plan": output.action.steps,
                "current_step": 0,
            }
    
    return replanner


print("Replanner node creator defined!")

## Step 6: Define Routing Logic

In [ ]:
def route_after_executor(state: PlanExecuteState) -> str:
    """Route after executor: continue executing or move to replanner."""
    current_step = state.get("current_step", 0)
    plan = state.get("plan", [])
    
    if current_step < len(plan):
        # More steps to execute
        return "executor"
    else:
        # Plan complete, go to replanner
        return "replanner"


def route_after_replanner(state: PlanExecuteState) -> str:
    """Route after replanner: finalize or continue with new plan."""
    response = state.get("response", "")
    
    if response:
        # Response provided, we're done
        return "END"
    else:
        # New plan provided, continue executing
        return "executor"


print("Routing logic defined")

## Step 7: Build the Plan-Execute Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama

# Create LLM
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

# Build the graph
workflow = StateGraph(PlanExecuteState)

# Add nodes
workflow.add_node("planner", create_planner_node(llm))
workflow.add_node("executor", create_executor_node(llm))
workflow.add_node("replanner", create_replanner_node(llm))

# Build graph structure
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "executor")

# After executor: loop or go to replanner
workflow.add_conditional_edges(
    "executor",
    route_after_executor,
    {
        "executor": "executor",
        "replanner": "replanner",
    },
)

# After replanner: execute new plan or end
workflow.add_conditional_edges(
    "replanner",
    route_after_replanner,
    {
        "executor": "executor",
        "END": END,
    },
)

# Compile
graph = workflow.compile()

print("Plan-and-execute graph compiled!")
print("\nGraph flow:")
print("  START -> planner -> executor")
print("  executor -> executor (loop until plan done)")
print("  executor -> replanner (when plan complete)")
print("  replanner -> executor (new plan) OR END (response)")

## Step 8: Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render graph: {e}")

## Step 9: Run the Plan-Execute System

In [ ]:
def run_plan_execute(task: str):
    """Run a task through the plan-and-execute system."""
    
    print("="*70)
    print(f"Task: {task}")
    print("="*70)
    
    initial_state = {
        "task": task,
        "plan": [],
        "past_steps": [],
        "current_step": 0,
        "response": "",
    }
    
    result = graph.invoke(initial_state)
    
    print("\n" + "="*70)
    print("FINAL RESPONSE")
    print("="*70)
    print(result["response"])
    print("\n" + "="*70)
    print(f"Executed {len(result['past_steps'])} steps total")
    
    return result


# Test with a multi-step reasoning task
result = run_plan_execute(
    """Explain the key benefits of microservices architecture.
    Include at least 3 specific advantages and 2 potential challenges."""
)

## Step 10: Test with Different Task Types

Plan-and-execute excels at different types of complex tasks.

In [ ]:
# Research-style task
result2 = run_plan_execute(
    """Compare and contrast the economic systems of capitalism and socialism.
    Analyze their key principles, advantages, and real-world implementations."""
)

In [ ]:
# Creative task with multiple constraints
result3 = run_plan_execute(
    """Write a short story about a robot learning to paint.
    The story must:
    1. Be exactly 3 paragraphs
    2. Include a plot twist
    3. End with a meaningful lesson about creativity"""
)

## Step 11: Add Tools for Enhanced Execution

The executor can use tools via ReAct pattern for tasks requiring external information.

In [ ]:
from langchain_core.tools import tool


@tool
def search_knowledge_base(query: str) -> str:
    """Search a knowledge base for information.
    
    Args:
        query: The search query
    
    Returns:
        Relevant information from the knowledge base
    """
    # Mock implementation
    knowledge = {
        "python": "Python is a high-level, interpreted programming language known for readability and simplicity.",
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications with LLMs using graphs.",
        "agents": "Agents are systems that can perceive their environment and take actions to achieve goals.",
    }
    
    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return value
    
    return "No information found for this query."


@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression.
    
    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 2", "10 * 5")
    
    Returns:
        The result of the calculation
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


tools = [search_knowledge_base, calculate]

# Create graph with tools
workflow_with_tools = StateGraph(PlanExecuteState)
workflow_with_tools.add_node("planner", create_planner_node(llm))
workflow_with_tools.add_node("executor", create_executor_node(llm, tools=tools))
workflow_with_tools.add_node("replanner", create_replanner_node(llm))

workflow_with_tools.add_edge(START, "planner")
workflow_with_tools.add_edge("planner", "executor")
workflow_with_tools.add_conditional_edges(
    "executor",
    route_after_executor,
    {"executor": "executor", "replanner": "replanner"},
)
workflow_with_tools.add_conditional_edges(
    "replanner",
    route_after_replanner,
    {"executor": "executor", "END": END},
)

graph_with_tools = workflow_with_tools.compile()

print("Graph with tools compiled!")
print(f"Available tools: {[t.name for t in tools]}")

In [ ]:
# Test with tools
def run_with_tools(task: str):
    initial_state = {
        "task": task,
        "plan": [],
        "past_steps": [],
        "current_step": 0,
        "response": "",
    }
    result = graph_with_tools.invoke(initial_state)
    print("\nFinal Response:")
    print(result["response"])
    return result


result4 = run_with_tools(
    """First, search for information about LangGraph.
    Then calculate how many graphs you could build in 100 hours if each takes 5 hours.
    Finally, explain how LangGraph helps in building those graphs efficiently."""
)

## Step 12: Using the Built-in Module

The `langgraph_ollama_local.patterns` module provides a ready-to-use implementation:

In [ ]:
from langgraph_ollama_local.patterns import (
    create_plan_execute_graph,
    run_plan_execute_task,
)

# Create the graph using the module
module_graph = create_plan_execute_graph(llm, tools=tools)

# Run a task
result = run_plan_execute_task(
    module_graph,
    """Analyze the impact of remote work on three areas:
    1) Employee productivity
    2) Company culture
    3) Urban development patterns"""
)

print("\nFinal Response:")
print(result["response"])

## Step 13: Complete Code

Here's the complete implementation in one cell for reference:

In [ ]:
# === Complete Plan-and-Execute Implementation ===

from typing import Annotated, Union
from typing_extensions import TypedDict
import operator

from langchain_core.messages import SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field

from langgraph_ollama_local import LocalAgentConfig


# === State ===
class PlanExecuteState(TypedDict):
    task: str
    plan: list[str]
    past_steps: Annotated[list[tuple[str, str]], operator.add]
    current_step: int
    response: str


# === Structured Outputs ===
class Plan(BaseModel):
    steps: list[str] = Field(description="List of steps")


class Response(BaseModel):
    response: str = Field(description="Final response")


class Act(BaseModel):
    action: Union[Response, Plan] = Field(description="Response or new Plan")


# === Node Creators ===
def create_planner(llm):
    structured_llm = llm.with_structured_output(Plan)
    def planner(state):
        output = structured_llm.invoke([
            HumanMessage(content=f"Create 3-7 step plan for: {state['task']}")
        ])
        return {"plan": output.steps, "current_step": 0}
    return planner


def create_executor(llm):
    def executor(state):
        if state["current_step"] >= len(state["plan"]):
            return {}
        step = state["plan"][state["current_step"]]
        response = llm.invoke([HumanMessage(content=f"Execute: {step}")])
        return {
            "past_steps": [(step, response.content)],
            "current_step": state["current_step"] + 1,
        }
    return executor


def create_replanner(llm):
    structured_llm = llm.with_structured_output(Act)
    def replanner(state):
        summary = "\n".join([f"{s}: {r[:100]}" for s, r in state["past_steps"]])
        output = structured_llm.invoke([
            HumanMessage(content=f"Task: {state['task']}\nCompleted:\n{summary}\n\nRespond or create new plan?")
        ])
        if isinstance(output.action, Response):
            return {"response": output.action.response}
        return {"plan": output.action.steps, "current_step": 0}
    return replanner


# === Routing ===
def route_executor(state):
    return "executor" if state["current_step"] < len(state["plan"]) else "replanner"


def route_replanner(state):
    return "END" if state.get("response") else "executor"


# === Build Graph ===
def build_plan_execute_graph():
    config = LocalAgentConfig()
    llm = ChatOllama(model=config.ollama.model, base_url=config.ollama.base_url, temperature=0)
    
    g = StateGraph(PlanExecuteState)
    g.add_node("planner", create_planner(llm))
    g.add_node("executor", create_executor(llm))
    g.add_node("replanner", create_replanner(llm))
    
    g.add_edge(START, "planner")
    g.add_edge("planner", "executor")
    g.add_conditional_edges("executor", route_executor, {"executor": "executor", "replanner": "replanner"})
    g.add_conditional_edges("replanner", route_replanner, {"executor": "executor", "END": END})
    
    return g.compile()


# === Use ===
if __name__ == "__main__":
    graph = build_plan_execute_graph()
    result = graph.invoke({
        "task": "Explain quantum computing in simple terms",
        "plan": [],
        "past_steps": [],
        "current_step": 0,
        "response": "",
    })
    print(result["response"])

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Planner** | Creates step-by-step plan upfront |
| **Executor** | Processes steps sequentially with context |
| **Replanner** | Adaptively decides to finalize or create new plan |
| **past_steps** | Accumulates (step, result) pairs for context |
| **Structured Output** | Pydantic models ensure reliable parsing |
| **Tool Integration** | Executor can use ReAct pattern with tools |
| **Two-Phase** | Separate planning from execution for efficiency |

## When to Use Plan-Execute vs Other Patterns

| Pattern | Best For | Planning | Execution | Adaptation |
|---------|----------|----------|-----------|------------|
| **Plan-Execute** | Multi-step tasks with dependencies | Upfront | Sequential | Replanning after completion |
| **ReAct** | Simple tasks, exploratory work | Step-by-step | Interleaved | Continuous |
| **ReWOO** | Known workflows, token efficiency | Complete upfront | Parallel | None |
| **Reflection** | Quality improvement, iterative refinement | None | Single output | Critique-revise loop |

**Choose plan-execute when:**
- Task requires multiple coordinated steps
- You want visibility into the strategy before execution
- Different models for planning vs execution would be beneficial
- Task complexity justifies upfront planning overhead

**Avoid plan-execute when:**
- Task is simple and straightforward
- Exploration and discovery are key
- Steps can't be determined upfront
- Real-time adaptation is more important than planning

## What's Next

You've mastered the plan-and-execute pattern! You now know:
- How to create step-by-step plans from complex tasks
- Execute plans sequentially with context from previous steps
- Implement adaptive replanning based on results
- Integrate tools for enhanced execution capabilities

**Continue with Advanced Reasoning:**
- **Tutorial 22**: Reflection - Iterative improvement through critique
- **Tutorial 23**: Reflexion - Learning from failures across attempts
- **Tutorial 24**: LATS - Monte Carlo Tree Search for agents
- **Tutorial 25**: ReWOO - Decoupled planning with parallel execution

Each pattern offers unique advantages for different types of complex reasoning tasks!